<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/GoogleQA_tfhub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Dec 24 21:40:05 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# bert uncased base
!wget https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1?tf-hub-format=compressed -O bbu
!mkdir bert_base_uncased
!tar -xzf bbu -C bert_base_uncased

# bert cased base
!wget https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1?tf-hub-format=compressed -O bbc
!mkdir bert_base_cased
!tar -xzf bbc -C bert_base_cased

# albert base
!wget https://tfhub.dev/google/albert_base/1?tf-hub-format=compressed -O ab
!mkdir albert_base
!tar -xzf ab -C albert_base

# bert uncased large
!wget https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1?tf-hub-format=compressed -O blu
!mkdir bert_large_uncased
!tar -xzf blu -C bert_large_uncased

--2019-12-24 21:40:09--  https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1?tf-hub-format=compressed
Resolving tfhub.dev (tfhub.dev)... 74.125.20.113, 74.125.20.138, 74.125.20.139, ...
Connecting to tfhub.dev (tfhub.dev)|74.125.20.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/tfhub-modules/tensorflow/bert_en_uncased_L-12_H-768_A-12/1.tar.gz [following]
--2019-12-24 21:40:14--  https://storage.googleapis.com/tfhub-modules/tensorflow/bert_en_uncased_L-12_H-768_A-12/1.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405800905 (387M) [application/x-tar]
Saving to: ‘bbu’

bbu                 100%[===================>] 387.00M   122MB/s    in 3.2s    

2019-12-24 21:40:18 (122 MB/s) - ‘bbu’ saved [405800905/

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
%tensorflow_version 2.x
import sys
sys.path.insert(1, '/content/drive/My Drive/GoogleQA')
import numpy as np
import pandas as pd
import os, gc
import codecs
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.backend import set_session
# from keras.callbacks import Callback
# from keras.models import Model, load_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, GroupKFold, KFold, RepeatedKFold
from scipy.stats import spearmanr
from tqdm import tqdm
import tensorflow_hub as hub
# from albert_tokenization import FullTokenizer
from tokenizer import Tokenizer
from warmup_v2 import AdamWarmup, calc_train_steps
tf.get_logger().setLevel('ERROR') 
# from keras_bert import load_trained_model_from_checkpoint, Tokenizer, AdamWarmup, calc_train_steps, get_custom_objects
from hyperopt import fmin, hp, tpe, STATUS_OK, Trials
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0-rc1


In [0]:
data_path = 'drive/My Drive/GoogleQA/Data/'
aux_path = 'drive/My Drive/GoogleQA/Web Scrap/'
train = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')
sub = pd.read_csv(data_path+'sample_submission.csv')
# qa_aux = pd.read_csv(aux_path+'df_qa_extracted.csv')[['qa_id', 'question_viewed_times', 'question_votes', 'answer_counts', 'answer_votes']]
# quser_aux = pd.read_csv(aux_path+'df_question_user_extracted.csv')
# auser_aux = pd.read_csv(aux_path+'df_answer_user_extracted.csv')
target_col = train.columns.tolist()[11:42]

train['class'] = train.host.apply(lambda x: x.split('.')[0])
train['question'] = train['category']+' '+train['class']+' '+train['question_title']+' '+train['question_body']
     

test['class'] = test.host.apply(lambda x: x.split('.')[0])
test['question'] = test['category']+' '+test['class']+' '+test['question_title']+' '+test['question_body']
 




In [0]:
MAXLEN = 512 #@param {type:"slider", min:128, max:512, step:32}
BATCH_SIZE = 4 #@param {type:'slider', min:1, max:32, step:1}
NUM_EPOCHS = 3
NUM_CLASSES = 30
NUM_AUX = 6
LR = 5e-5
MIN_LR = 0
model_path = 'bert_base_uncased' #@param ['bert_base_uncased', 'bert_base_cased', 'bert_large_uncased', 'albert_base']
save_path = 'tmp'
save_model = 'bbu5' 
CASED = False #@param ['True', 'False'] {type:"raw"}
# OUTPUT_TRAIN = 'train_bert_ipredcv1415_oof.csv'
# OUTPUT_TEST = 'test_bert_large.npy'
# model_path = 'uncased_L-12_H-768_A-12' #@param ['uncased_L-12_H-768_A-12', 'wwm_uncased_L-24_H-1024_A-16', 'uncased_L-24_H-1024_A-16']
# target_q_col = train.columns.tolist()[11:32]
# target_a_col = train.columns.tolist()[32:42]


## Bert tokenizer

In [0]:
token_dict = {}
with codecs.open(os.path.join(model_path, 'assets/vocab.txt'), 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
tokenizer = Tokenizer(token_dict, cased=CASED)

def convert_data(data_df, branch='training'):
    data_df = data_df.reset_index(drop=True)
    global tokenizer
    global MAXLEN
    global target_col
    ids, segments = [], []
    # q_title, q_body, answer = [], [], []
    for i in tqdm(range(len(data_df))):
        a, _ = tokenizer.encode(data_df.loc[i, 'question'])
        b, _ = tokenizer.encode(data_df.loc[i, 'answer'])
        if len(a) + len(b) >512:
            if (len(a) > 256) & (len(b) > 256):
                a, b = a[:256], b[:256]
            elif len(a) > len(b):
                a = a[:(512-len(b))]
            elif len(a) <= len(b):
                b = b[:(512-len(a))]
        ids.append(a + b)
        segments.append([0] * len(a) + [1] * len(b))
    # aux = data_df[['question_viewed_times', 'question_votes', 'answer_counts', 'answer_votes', 'answer_user_reputation', 'question_user_reputation']]
    # aux = aux.apply(lambda x: (x - x.mean()) / x.std())             
    if branch == 'training':
        targets = data_df[target_col]
        return [ids, segments], np.array(targets)
        # return [ids, segments], np.array(aux, dtype='float32'), np.array(targets)
    else:
        # return [ids, segments], np.array(aux, dtype='float32')
        return [ids, segments]

In [0]:
"""## Data Generator"""
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = min(max(L), MAXLEN)
    # ML = MAXLEN
    out = np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x[:ML] for x in X])
    return tf.convert_to_tensor(out, dtype=tf.int32)
    
def get_masks(X, padding=0):
    L = [len(x) for x in X]
    ML = min(max(L), MAXLEN)
    # ML = MAXLEN
    out = np.array([np.concatenate([[1]*len(x), [padding] * (ML - len(x))]) if len(x) < ML else [1]*ML for x in X])
    return tf.convert_to_tensor(out, dtype=tf.int32)

class data_generator:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train'):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        # self.q_a = q_a
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                X1 = seq_padding([x[0] for x in d])
                X2 = get_masks([x[0] for x in d])    
                X3 = seq_padding([x[1] for x in d])
                # X3 = np.zeros_like(X1)

                aux = tf.convert_to_tensor([x[2] for x in d], dtype=tf.float32)
                if self.branch == 'test':
                    # aux = np.array([x[3] for x in d])
                    yield [X1, X2, X3]
                    # yield [X1, X2, X3, X4, X5, X6]
                else:
                    Y = tf.convert_to_tensor([x[2] for x in d], dtype=tf.float32)
                    # aux = np.array([x[4] for x in d])
                    yield [X1, X2, X3], Y
                    # yield [X1, X2, X3, X4, X5, X6], Y


In [0]:
def model_build(len_train):
    global NUM_CLASSES
    global BATCH_SIZE
    global NUM_EPOCHS
    global MIN_LR
    global LR
    global MAXLEN
    global model_path
    global NUM_AUX
 
    q_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_input_word_ids")
    q2_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_input_masks")
    q3_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_segment_ids")


    bert_layer = hub.KerasLayer(model_path, trainable=True)
    _, q_inputs  = bert_layer([q_in, q2_in, q3_in])
    # q_outputs = keras.layers.Lambda(lambda x: x[:, 0])(q_inputs)
    q_outputs = keras.layers.GlobalAveragePooling1D()(q_inputs)
    dense = keras.layers.Dropout(0.1)(q_outputs)

    # dense = keras.layers.concatenate([dense, aux_in])
    outputs = keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dense)
    # aux_out = keras.layers.Dense(NUM_AUX)(dense)
    model = keras.Model([q_in, q2_in, q3_in], [outputs])

    decay_steps, warmup_steps = calc_train_steps(
        len_train,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
    )

    model.compile(
        loss=['binary_crossentropy'],
        # loss_weights = [10, 0.25],
        optimizer=AdamWarmup(
            decay_steps=decay_steps,
            warmup_steps=warmup_steps,
            lr=LR,
            min_lr=MIN_LR,
            ))
    #      optimizer=keras.optimizers.Adam(LR),
    # )

    return model

## train model

In [0]:
# pred = np.zeros((len(test), NUM_CLASSES))
kf = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1627)
idx = [x for x in kf.split(train, groups=train.question_body)]
# tr1, tr2 = train.loc[idx[0][0]], train.loc[idx[0][1]]
# tr1 = tr1.reset_index(drop=True)
# idx1 = [x for x in kf.split(tr1, groups=tr1.question_body)]


def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred).correlation)
    return np.mean(rhos)

class IntervalEval(keras.callbacks.Callback):
    def __init__(self):
        global NUM_EPOCHS
        global save_model
        super(keras.callbacks.Callback, self).__init__()
    def on_epoch_end(self, epoch, logs={}):
        val_pred = self.model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
        # pred = np.floor(val_pred[0] * 90) / 90
        score = compute_spearmanr(val_y, val_pred)
        print('Spearman - {:.5f}'.format(score))
        if epoch>=NUM_EPOCHS-1:
            print('--Save Model--')
            self.model.save('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}-{:}.h5'.format(save_path, save_model, i, epoch+1))


In [14]:
for i, (tr_idx, val_idx) in enumerate(idx, 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    # tr_aux, val_aux = train_aux.loc[tr_idx], train_aux.loc[val_idx]
    tr_x, tr_y = convert_data(tr)
    val_x, val_y = convert_data(val)
    
    model = model_build(len(tr))

    train_D = data_generator(list(zip(tr_x[0], tr_x[1], tr_y)))
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_y)), branch='valid')
    # test_D = data_generator(list(zip(test_x[0], test_x[1])), branch='test')
    ieval = IntervalEval()
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        callbacks = [ieval]
    )

    gc.collect()

  1%|          | 28/5471 [00:00<00:22, 241.55it/s]


Fold - 1



100%|██████████| 608/608 [00:01<00:00, 309.19it/s]


Train for 1368 steps
Epoch 1/3
152/152 [==============================] - 15s 99ms/step
Spearman - 0.38144
1368/1368 [==============================] - 461s 337ms/step - loss: 0.3986
Epoch 2/3
152/152 [==============================] - 14s 93ms/step
Spearman - 0.40211
1368/1368 [==============================] - 427s 312ms/step - loss: 0.3597
Epoch 3/3
152/152 [==============================] - 14s 92ms/step
Spearman - 0.40181
--Save Model--
1368/1368 [==============================] - 432s 316ms/step - loss: 0.3349


  1%|          | 30/5471 [00:00<00:20, 266.42it/s]

...

Train for 1368 steps
Epoch 1/3
152/152 [==============================] - 15s 99ms/step
Spearman - 0.36831
1368/1368 [==============================] - 456s 333ms/step - loss: 0.4022
Epoch 2/3
152/152 [==============================] - 14s 92ms/step
Spearman - 0.39010
1368/1368 [==============================] - 426s 311ms/step - loss: 0.3605
Epoch 3/3
152/152 [==============================] - 14s 92ms/step
Spearman - 0.39625
--Save Model--
1368/1368 [==============================] - 433s 317ms/step - loss: 0.3348


  1%|          | 31/5472 [00:00<00:19, 277.95it/s]


Fold - 10



100%|██████████| 607/607 [00:09<00:00, 67.29it/s] 


Train for 1368 steps
Epoch 1/3
152/152 [==============================] - 15s 100ms/step
Spearman - 0.38546
1368/1368 [==============================] - 455s 333ms/step - loss: 0.4022
Epoch 2/3
152/152 [==============================] - 14s 94ms/step
Spearman - 0.41034
1368/1368 [==============================] - 425s 311ms/step - loss: 0.3610
Epoch 3/3
152/152 [==============================] - 14s 94ms/step
Spearman - 0.40808
--Save Model--
1368/1368 [==============================] - 434s 317ms/step - loss: 0.3357


## oof-prediction

In [0]:
model = model_build(len(train))
oof_pred = train.copy()
oof_pred[target_col] = 0

In [0]:
for i, (tr_idx, val_idx) in enumerate(idx, 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]

    val_x, val_y = convert_data(val)
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_y)), branch='valid')

    model.load_weights('drive/My Drive/GoogleQA/Models/{}/{}-{}-3.h5'.format(save_path, save_model, i))
    val_pred = model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
    oof_pred.loc[val_idx, target_col] += val_pred #/2

    
    score = compute_spearmanr(val_y, val_pred)
    print('Spearman - {:.5f}'.format(score))

    gc.collect()

oof_pred.to_csv('drive/My Drive/GoogleQA/Data/train_oof_pred_bbu5.csv', index=False)

  5%|▍         | 29/608 [00:00<00:01, 289.63it/s]


Fold - 1



100%|██████████| 608/608 [00:01<00:00, 310.82it/s]


152/152 [==============================] - 17s 109ms/step
Spearman - 0.40181

  6%|▌         | 37/608 [00:00<00:01, 368.22it/s]

...

100%|██████████| 608/608 [00:01<00:00, 324.18it/s]


152/152 [==============================] - 14s 91ms/step
Spearman - 0.39973


  0%|          | 0/608 [00:00<?, ?it/s]


Fold - 6



100%|██████████| 608/608 [00:01<00:00, 334.49it/s]


In [0]:
# bbu3: 0.42130
# bbu5: 
compute_spearmanr(np.floor(oof_pred[target_col].values*90)/90, train[target_col].values)

0.4213022239256377

In [0]:
oof_pred = pd.read_csv('drive/My Drive/GoogleQA/Data/train_oof_pred.csv')

In [0]:
corrparams = {'p': hp.randint('p', len(oof_pred))}
from functools import partial


def f_min(params, col):
    tmp = oof_pred.copy()[col].values
    # colidx = target_col.index(col)
    rowidx = tmp.argsort()[:params['p']]
    tmp[rowidx] = 0
    score = spearmanr(train[col].values, tmp).correlation
    return {'loss': -round(score, 5), 'status': STATUS_OK}

score = []
for col in target_col:
    f = partial(f_min, col=col)
    trials = Trials()
    best = fmin(f, corrparams, tpe.suggest, 100, rstate=np.random.RandomState(0), trials=trials, show_progressbar=False)
    print('{} - {} - {:.2f}'.format(col, -trials.best_trial['result']['loss'], best['p']/len(oof_pred)))
    score.append(-trials.best_trial['result']['loss'])



question_asker_intent_understanding - 0.37869 - 0.00
question_body_critical - 0.62999 - 0.00
question_conversational - 0.50808 - 0.91
question_expect_short_answer - 0.30653 - 0.00
question_fact_seeking - 0.37795 - 0.00
question_has_commonly_accepted_answer - 0.43624 - 0.00
question_interestingness_others - 0.34862 - 0.24
question_interestingness_self - 0.50938 - 0.53
question_multi_intent - 0.58301 - 0.40
question_not_really_a_question - 0.10707 - 0.88
question_opinion_seeking - 0.48817 - 0.04
question_type_choice - 0.75155 - 0.48
question_type_compare - 0.53939 - 0.95
question_type_consequence - 0.25704 - 0.99
question_type_definition - 0.63655 - 0.93
question_type_entity - 0.61179 - 0.88
question_type_instructions - 0.78876 - 0.26
question_type_procedure - 0.35947 - 0.07
question_type_reason_explanation - 0.67579 - 0.20
question_type_spelling - 0.16542 - 1.00
question_well_written - 0.52394 - 0.09
answer_helpful - 0.23555 - 0.00
answer_level_of_information - 0.41974 - 0.00
answer_pla

In [0]:
print(np.mean(score))

0.441695


In [0]:
for col, col_trues, col_pred in zip(target_col, train[target_col].values.T, oof_pred[target_col].values.T):
    corr = spearmanr(col_trues, col_pred).correlation
    print('{} - {:.5f}'.format(col, corr))

question_asker_intent_understanding - 0.37869
question_body_critical - 0.62999
question_conversational - 0.41516
question_expect_short_answer - 0.30652
question_fact_seeking - 0.37796
question_has_commonly_accepted_answer - 0.43624
question_interestingness_others - 0.34694
question_interestingness_self - 0.49711
question_multi_intent - 0.57917
question_not_really_a_question - 0.07784
question_opinion_seeking - 0.48814
question_type_choice - 0.73322
question_type_compare - 0.36039
question_type_consequence - 0.18191
question_type_definition - 0.36164
question_type_entity - 0.46276
question_type_instructions - 0.78688
question_type_procedure - 0.35943
question_type_reason_explanation - 0.67554
question_type_spelling - 0.06805
question_well_written - 0.52352
answer_helpful - 0.23555
answer_level_of_information - 0.41975
answer_plausible - 0.16574
answer_relevance - 0.18567
answer_satisfaction - 0.33193
answer_type_instructions - 0.76661
answer_type_procedure - 0.28892
answer_type_reason_e

In [0]:
for col, col_trues, col_pred in zip(target_col, train[target_col].values.T, (np.floor(oof_pred[target_col].values *90)/90).T):
    corr = spearmanr(col_trues, col_pred).correlation
    print('{} - {:.5f}'.format(col, corr))

question_asker_intent_understanding - 0.37761
question_body_critical - 0.63009
question_conversational - 0.44532
question_expect_short_answer - 0.30656
question_fact_seeking - 0.37719
question_has_commonly_accepted_answer - 0.43661
question_interestingness_others - 0.34643
question_interestingness_self - 0.49692
question_multi_intent - 0.57949
question_not_really_a_question - 0.10288
question_opinion_seeking - 0.48791
question_type_choice - 0.73357
question_type_compare - 0.39868
question_type_consequence - 0.20775
question_type_definition - 0.47134
question_type_entity - 0.48330
question_type_instructions - 0.78705
question_type_procedure - 0.35963
question_type_reason_explanation - 0.67561
question_type_spelling - 0.15789
question_well_written - 0.52359
answer_helpful - 0.23444
answer_level_of_information - 0.41842
answer_plausible - 0.16095
answer_relevance - 0.18324
answer_satisfaction - 0.32933
answer_type_instructions - 0.76696
answer_type_procedure - 0.28885
answer_type_reason_e

## test prediction

In [0]:
test_pred = pd.read_csv('drive/My Drive/GoogleQA/Data/test_oof_pred.csv')
hyper = pd.read_csv('drive/My Drive/GoogleQA/Data/hyper.csv')
pred = test_pred[target_col].copy().values

,col,pct
0,question_conversational,0.91
1,question_interestingness_others,0.24
2,question_interestingness_self,0.53
3,question_multi_intent,0.40
4,question_not_really_a_question,0.88
5,question_opinion_seeking,0.04
6,question_type_choice,0.48
7,question_type_compare,0.95
8,question_type_consequence,0.99
9,question_type_definition,0.93


In [0]:
for col in [x for x in target_col if x not in hyper.col.tolist()]:
    pred[:, target_col.index(col)] = (pred[:, target_col.index(col)]//(1/90))/90
for col in hyper.col.tolist():
    if hyper.loc[hyper.col==col, 'pct'].values == 1:
        pct = hyper.loc[hyper.col==col, 'pct'] - 0.01
    else:
        pct = hyper.loc[hyper.col==col, 'pct']
    changerow = int(len(test) * pct)
    rowidx = pred[:, target_col.index(col)].argsort()[:changerow]
    pred[rowidx, target_col.index(col)] = 0



19    0.17
Name: pct, dtype: float64

In [0]:
for i in range(pred.shape[1]):
    print(i, len(np.unique(pred[:, i])))

0 24
1 53
2 44
3 58
4 49
5 54
6 363
7 225
8 287
9 59
10 458
11 249
12 25
13 6
14 35
15 59
16 354
17 444
18 382
19 6
20 435
21 16
22 25
23 8
24 9
25 24
26 363
27 458
28 439
29 397


## Bert model stacking

In [0]:
# 0.8771 / 0.8779 / 0.8786 / 0.8774 / 0.8790
tr2_x, tr2_y = convert_data(tr2)
valid2_D = data_generator(list(zip(tr2_x[0], tr2_x[1], tr2_y)), branch='valid')
model = model_build(len(train), model='bert')
ensemble_inputs, ensemble_outputs = [], []
# update all layers in all models to not be trainable
for i in range(1, 11):
    # define filename for this ensemble
    filename = 'drive/My Drive/GoogleQA/Models/{:}/{:}-{:}-3.h5'.format(save_model, save_model, i))

    # load model from file
    model.load_weights(filename)
    print('>loaded %s' % filename)

    model_pred = model.predict_generator(valid2_D.__iter__(), len(valid2_D), verbose=1)
    score = compute_spearmanr(tr2_y, model_pred)
    print('Spearman - {:.5f}'.format(score))
    for layer in model.layers:
        # make not trainable
        layer.trainable = False
        # rename to avoid 'unique layer name' issue
        layer.name = 'ensemble_' + str(i+1) + '_' + layer.name
    # define multi-headed input
    ensemble_inputs.append(model.input)
    # concatenate merge output from each model
    ensemble_outputs.append(model.output)
    # del model
    gc.collect()

# build model
merge = concatenate(ensemble_outputs)
hidden = Dense(100, activation='relu')(merge)
output = Dense(NUM_CLASSES, activation='softmax')(hidden)
stacked_model = Model(inputs=[x for y in ensemble_inputs for x in y], outputs=output)
# compile
decay_steps, warmup_steps = calc_train_steps(
    len(tr2),
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
)

stacked_model.compile(
    loss='binary_crossentropy',
    optimizer=AdamWarmup(
        decay_steps=decay_steps,
        warmup_steps=warmup_steps,
        lr=LR,
        min_lr=MIN_LR,
        ))



In [0]:
class data_generator_stacked:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train'):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                X1 = seq_padding([x[0] for x in d])
                X2 = get_masks([x[0] for x in d])    
                X3 = seq_padding([x[1] for x in d])
                # X3 = np.zeros_like(X1)

                # aux = np.array([x[2] for x in d])
                if self.branch == 'test':
                    # aux = np.array([x[3] for x in d])
                    yield [X1, X2, X3] * 10
                    # yield [X1, X2, X3, X4, X5, X6]
                else:
                    Y = np.array([x[2] for x in d])
                    # aux = np.array([x[4] for x in d])
                    yield [X1, X2, X3] * 10, Y
                    # yield [X1, X2, X3, X4, X5, X6], Y

# fit model
stack_D = data_generator_stacked(list(zip(tr2_x[0], tr2_x[1], tr2_y)))
stacked_model.fit_generator(
    stack_D.__iter__(),
    steps_per_epoch=len(stack_D),
    epochs=NUM_EPOCHS)

stack_valid_D = data_generator_stacked(list(zip(tr2_x[0], tr2_x[1], tr2_y)), branch='valid')
stacked_pred = stacked_model.predict_generator(stack_valid_D.__iter__(), len(stack_valid_D), verbose=1)
print('Spearman - {:.5f}'.format(compute_spearmanr(tr2_y, stacked_pred)))

del stacked_model
gc.collect()

## USE Model

In [0]:

# train['sentence'] = train['question'] + ' ' + train['answer']
embed1 = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
# embed2 = hub.load('https://tfhub.dev/google/nnlm-en-dim128/2')

def UniversalEmbedding(x):
    return embed1(tf.squeeze(tf.cast(x, tf.string)))

# def NNLMEmbedding(x):
#     return embed2(tf.squeeze(tf.cast(x, tf.string)))

def use_model():
    
    q = keras.layers.Input(shape=(1,), dtype=tf.string)
    qb = keras.layers.Input(shape=(1,), dtype=tf.string)
    a = keras.layers.Input(shape=(1,), dtype=tf.string)

    def hidden_layer(input_layer):
        x = keras.layers.Lambda(UniversalEmbedding, output_shape=(512, ))(input_layer)
        # embed2 = keras.layers.Lambda(NNLMEmbedding, output_shape=(128, ))(input_layer)
        # embed = keras.layers.concatenate([embed1, embed2])
        x = keras.layers.Dropout(0.2)(x)
        x = keras.backend.expand_dims(x, axis=1)
        # x = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(128, return_sequences=True))(x)
        # x = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(128, return_sequences=True))(x)
        # GP = keras.layers.GlobalMaxPooling1D()(x)
        # GP = keras.layers.Dropout(0.2)(GP)
        # AP = keras.layers.GlobalAveragePooling1D()(x)
        # AP = keras.layers.Dropout(0.2)(AP)
        # hidden = keras.layers.concatenate([GP, AP])
        # hidden = keras.layers.add([hidden, keras.layers.Dense(512, activation='relu')(hidden)])
        # hidden = keras.layers.add([hidden, keras.layers.Dense(512, activation='relu')(hidden)])

        x1 = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(256, return_sequences=True))(x)
        x2 = keras.layers.Bidirectional(keras.layers.CuDNNGRU(128, return_sequences=True))(x1)
        max_pool1 = keras.layers.GlobalMaxPooling1D()(x1)
        max_pool2 = keras.layers.GlobalMaxPooling1D()(x2)
        hidden = keras.layers.concatenate([max_pool1, max_pool2])
        return hidden

    q_embed, qb_embed, a_embed = hidden_layer(q), hidden_layer(qb), hidden_layer(a)
    all_embed = keras.layers.concatenate([q_embed, qb_embed, a_embed])
    dense = keras.layers.Dense(256, activation='relu')(all_embed)
    pred = keras.layers.Dense(30, activation='sigmoid')(dense)
    model = keras.Model(inputs=[q, qb, a], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=5e-3))
    # del embed
    # gc.collect()
    return model


In [0]:
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=0)
idx = [x for x in kf.split(train, groups=train.question_body)]
def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred).correlation)
    return np.mean(rhos)

class IntervalEval(keras.callbacks.Callback):
    def __init__(self, val_data, label):
        super(keras.callbacks.Callback, self).__init__()
        self.val_data = val_data
        self.score = 0
        self.maxscore = 0
        self.label = label
        self.patience = 3
        self.count = 0
    def on_epoch_end(self, epoch, logs={}):
        # if epoch == 3:
        val_pred = self.model.predict(self.val_data, batch_size=16, verbose=1)
        score = compute_spearmanr(self.label, val_pred)
        print('Spearman - {:.5f}'.format(score))
        if self.maxscore>=0.3:
            self.patience=2
        if (score < self.score):
            self.score = score
            self.count+=1
            if self.count==self.patience:
                self.model.stop_training=True
        elif score > self.maxscore:
            self.score = score
            self.maxscore = score
            self.count = 0
            if self.maxscore > 0.3:
                self.model.save('drive/My Drive/GoogleQA/Models/{:}/use-model-{:}.h5'.format(save_model, i))
        else:
            self.score = score
            self.count = 0
          

In [0]:
with tf.Session() as session:
    keras.backend.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    for i, (tr_idx, val_idx) in enumerate(idx, 1):
        print('\nFold - {:}\n'.format(i))
        tr, val = train.loc[tr_idx], train.loc[val_idx]
        tr_x = [tr['question_title'], tr['question_body'], tr['answer']]
        val_x = [val['question_title'], val['question_body'], val['answer']]
        ieval = IntervalEval(val_data=val_x, label=val[target_col].values)
        model = use_model()
        history = model.fit(
            tr_x,
            tr[target_col],
            epochs=100, 
            batch_size=16,
            callbacks=[ieval])
        del model, tr, val, tr_x, val_x
        gc.collect()

# with tf.Session() as session:
#     keras.backend.set_session(session)
#     session.run(tf.global_variables_initializer())  
#     session.run(tf.tables_initializer())
#     model.load_weights('use-model-1.h5')
#     preds = model.predict(val['sentence'], batch_size=16, verbose=1)
#     score=compute_spearmanr(val[target_col].values, preds)
#     print('Spearman - {:.5f}'.format(score))


Fold - 1

Train on 4863 samples
Epoch 1/100
1216/1216 [==============================] - 33s 27ms/sample
Spearman - 0.35849
4863/4863 [==============================] - 170s 35ms/sample - loss: 0.3946
Epoch 2/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.36453
4863/4863 [==============================] - 85s 18ms/sample - loss: 0.3720
Epoch 3/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37807
4863/4863 [==============================] - 85s 18ms/sample - loss: 0.3628
Epoch 4/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37832
4863/4863 [==============================] - 86s 18ms/sample - loss: 0.3549
Epoch 5/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37996
4863/4863 [==============================] - 86s 18ms/sample - loss: 0.3462
Epoch 6/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37266
4863/4863 [=======================